In [2]:
from copy import deepcopy
import numpy as np
import torch
from torch.optim import Adam
import gym
import time
import core
import torch.nn as nn

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import gymnasium as gym
import gym_examples
import envelope

In [4]:
class ReplayBuffer:
    """
    A simple FIFO experience replay buffer for DDPG agents.
    """

    def __init__(self, obs_dim, act_dim, size):
        self.obs_buf = np.zeros(core.combined_shape(size, obs_dim), dtype=np.float32)
        self.obs2_buf = np.zeros(core.combined_shape(size, obs_dim), dtype=np.float32)
        self.act_buf = np.zeros(core.combined_shape(size, act_dim), dtype=np.float32)
        self.rew_buf = np.zeros(size, dtype=np.float32)
        self.done_buf = np.zeros(size, dtype=np.float32)
        self.ptr, self.size, self.max_size = 0, 0, size

    def store(self, obs, act, rew, next_obs, done):
        self.obs_buf[self.ptr] = obs
        self.obs2_buf[self.ptr] = next_obs
        self.act_buf[self.ptr] = act
        self.rew_buf[self.ptr] = rew
        self.done_buf[self.ptr] = done
        self.ptr = (self.ptr+1) % self.max_size
        self.size = min(self.size+1, self.max_size)

    def sample_batch(self, batch_size=32):
        idxs = np.random.randint(0, self.size, size=batch_size) #batch_size 만큼 ReplayBuffer(D)에서 random sampling
        batch = dict(obs=self.obs_buf[idxs],
                     obs2=self.obs2_buf[idxs],
                     act=self.act_buf[idxs],
                     rew=self.rew_buf[idxs],
                     done=self.done_buf[idxs])
        return {k: torch.as_tensor(v, dtype=torch.float32) for k,v in batch.items()}

In [10]:
D = ReplayBuffer(obs_dim = 1000, act_dim = 1000, size = int(1e6))

In [12]:
B = D.sample_batch(batch_size=32)

ValueError: high <= 0

In [18]:
Fs = 10
N = 500
trans_info = [[{'coeff': 1.0, 'amps': 5 * np.array([-0.0154, -0.011, -0.005414]), 'freqs': [6.48e-3, 5.622e-3, 3.7e-3]},
               {'coeff': 0.01, 'amps': 5 * np.array([0.01237, -0.005816, -0.03928]), 'freqs': [0.01758, 0.005925, 0.002652]}],
              [{'coeff': -0.01, 'amps': 5 * np.array([-0.01568, -0.01486, -0.0064]), 'freqs': [0.00737, 0.0072, -0.0039]},
               {'coeff': 1.0, 'amps': 5 * np.array([-0.0154, -0.011, -0.005414]), 'freqs': [6.48e-3, 5.622e-3, 3.7e-3]}]]
A = envelope.flattop(t0=5.,len=10.,amp=1.,w=1.)
B = envelope.flattop(t0=15.,len=30.,amp=0.8,w=1.)
time = np.arange(N) / Fs  # (ns) 
freq = np.fft.fftfreq(N, d=1/Fs) # (GHz)
initial_state = np.hstack((A(time).astype('float32'),B(time).astype('float32'))) 

<ipython-input-18-20ba2ee0f664>:11: ComplexWarning: Casting complex values to real discards the imaginary part
  initial_state = np.hstack((A(time).astype('float32'),B(time).astype('float32')))


In [19]:
env = gym.make(id='gym_examples/Quantum', Fs=Fs, N=N, initial_state=initial_state,trans_info=trans_info)

In [40]:
o = env.reset()[0]

In [41]:
 a = env.action_space.sample()

In [42]:
o2, r, d, _, i = env.step(a)

In [43]:
o2.shape

(1000,)

In [44]:
D.store(o, a, r, o2, d)

In [45]:
test_obs_buff = np.zeros(core.combined_shape(int(1e6), env.observation_space.shape), dtype=np.float32)

In [46]:
type(test_obs_buff[0])

numpy.ndarray

In [47]:
type(o)

numpy.ndarray

In [48]:
type(a)

numpy.ndarray

In [39]:
type(o2)

numpy.ndarray